# keras conversion code

SNN 모델을 저장하되, IF 레이어는 ReLU로 바꿔서 저장하도록 한다. 대신 IF 레이어의 설정값은 따로 찍어서 보내기.

**주의점 : keras version 2.11이 들어간 환경에서 실행할 것! (KIST의 Neu+ 칩 구동환경과 일치시켜야 함)**

In [1]:
# path = '/data/hongwonseok/ECG-SNN/KIST/MLP/MLP_binary_NO_BatchNorm.pth'

path = '/home/hschoi/data/leehyunwon/ECG-SNN/SNN_MLP_ver5_poisson_binary_encoders187_hidden1000_encoderGradTrue_early30_lr0.001_threshold1.0_2024_10_02_14_12_25.pt'
 
 

In [2]:
import torch

model = torch.load(path)
model

/tmp/ipykernel_1375126/3090495766.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path)


{'epoch': 123,
 'model_state_dict': OrderedDict([('hidden.0.weight',
               tensor([[ 0.1393,  0.0648, -0.6093,  ..., -0.4128, -0.4444, -0.4954],
                       [-0.1112, -0.0165, -0.1261,  ...,  0.1066,  0.0200,  0.0537],
                       [ 0.4059,  0.1543,  0.3288,  ..., -0.0081, -0.0947,  0.0757],
                       ...,
                       [ 0.1602, -0.0882,  0.0658,  ...,  0.1971, -0.1087,  0.2221],
                       [-0.0941, -0.4773,  0.2361,  ..., -0.0926,  0.1224,  0.3461],
                       [ 0.4136,  0.0385, -0.6416,  ...,  0.3258,  0.3449, -0.2342]],
                      device='cuda:0')),
              ('layer.0.weight',
               tensor([[ 0.1423, -0.8289,  0.2361,  ..., -0.3478, -1.9131, -2.2378],
                       [-0.2553,  0.6203, -0.0082,  ...,  0.4773,  1.7018,  1.9360]],
                      device='cuda:0'))]),
 'optimizer_state_dict': {'state': {0: {'step': tensor(84816.),
    'exp_avg': tensor([[-3.0318e-05, -2.

In [3]:
weight = model['model_state_dict']
weight

OrderedDict([('hidden.0.weight',
              tensor([[ 0.1393,  0.0648, -0.6093,  ..., -0.4128, -0.4444, -0.4954],
                      [-0.1112, -0.0165, -0.1261,  ...,  0.1066,  0.0200,  0.0537],
                      [ 0.4059,  0.1543,  0.3288,  ..., -0.0081, -0.0947,  0.0757],
                      ...,
                      [ 0.1602, -0.0882,  0.0658,  ...,  0.1971, -0.1087,  0.2221],
                      [-0.0941, -0.4773,  0.2361,  ..., -0.0926,  0.1224,  0.3461],
                      [ 0.4136,  0.0385, -0.6416,  ...,  0.3258,  0.3449, -0.2342]],
                     device='cuda:0')),
             ('layer.0.weight',
              tensor([[ 0.1423, -0.8289,  0.2361,  ..., -0.3478, -1.9131, -2.2378],
                      [-0.2553,  0.6203, -0.0082,  ...,  0.4773,  1.7018,  1.9360]],
                     device='cuda:0'))])

In [4]:
weight.items()

odict_items([('hidden.0.weight', tensor([[ 0.1393,  0.0648, -0.6093,  ..., -0.4128, -0.4444, -0.4954],
        [-0.1112, -0.0165, -0.1261,  ...,  0.1066,  0.0200,  0.0537],
        [ 0.4059,  0.1543,  0.3288,  ..., -0.0081, -0.0947,  0.0757],
        ...,
        [ 0.1602, -0.0882,  0.0658,  ...,  0.1971, -0.1087,  0.2221],
        [-0.0941, -0.4773,  0.2361,  ..., -0.0926,  0.1224,  0.3461],
        [ 0.4136,  0.0385, -0.6416,  ...,  0.3258,  0.3449, -0.2342]],
       device='cuda:0')), ('layer.0.weight', tensor([[ 0.1423, -0.8289,  0.2361,  ..., -0.3478, -1.9131, -2.2378],
        [-0.2553,  0.6203, -0.0082,  ...,  0.4773,  1.7018,  1.9360]],
       device='cuda:0'))])

In [5]:
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout, InputLayer
from tensorflow.keras.layers import ZeroPadding1D, Conv1D, BatchNormalization, ReLU, AveragePooling1D, Flatten, Dense

2024-10-03 16:09:08.875890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


일단 GPT에게 변환 코드 예시를 받아놨으니 그걸로 진행해보자.

**KIST용으로 바꿀 땐 hidden layer 뉴런 수가 1000개 임에 주의!!!!**

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, ReLU

def create_keras_model_SNN(num_classes, num_encoders, hidden_size, hidden_size_2):
    # 입력 레이어 정의
    inputs = Input(shape=(num_encoders,), name='input')
    
    # 첫 번째 히든 레이어
    x = Dense(hidden_size, use_bias=False, name='hidden1')(inputs)
    x = ReLU(name='relu1')(x)
    
    # 히든레이어 하나만 써서 학습하는 모델로 변경
    # # 두 번째 히든 레이어
    # x = Dense(hidden_size_2, use_bias=False, name='hidden2')(x)
    # x = ReLU(name='relu2')(x)
    
    # 출력 레이어
    predictions = Dense(num_classes, use_bias=False, name='output')(x)
    predictions = ReLU(name='output_relu')(predictions)  # IFNode를 ReLU로 치환

    # 모델 생성
    model = Model(inputs=inputs, outputs=predictions)
    
    return model

# 모델 생성 예시
keras_model = create_keras_model_SNN(num_classes=2, num_encoders=187, hidden_size=1000, hidden_size_2=1000)

# 모델 구조 확인
keras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


2024-10-03 16:09:16.200102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 16:09:16.202017: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
# keras 모델의 초기 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[-0.00415822 -0.05357077  0.02445271 ... -0.00628012 -0.03349784
   0.01708192]
 [-0.01245295  0.07000272 -0.03547409 ... -0.06438427  0.0332396
  -0.03246547]
 [ 0.02798231  0.00014364  0.07093292 ... -0.0650633  -0.0515359
   0.00535967]
 ...
 [ 0.05727746  0.07037124  0.06633219 ...  0.0691597  -0.00310344
  -0.03603009]
 [ 0.04945737 -0.01611969 -0.04590061 ...  0.01937582 -0.01796092
   0.01494839]
 [-0.0201223  -0.03918933 -0.02204446 ... -0.01498163 -0.02500371
  -0.00871846]]
Layer: relu1
Layer: output
[[-0.03131597  0.05044855]
 [-0.01155195 -0.0363961 ]
 [-0.05490629 -0.02801292]
 ...
 [-0.0013093  -0.0464754 ]
 [-0.02287892  0.03038707]
 [-0.03344737  0.05387384]]
Layer: output_relu


가중치 추출하고 케라스 모델에 대입하기

왜인진 모르겠지만 cuda 커널 이미지가 안맞는다 하다가 또 된다고 하다가 그런다. 일단 시도해서 될 때 계속 진행하도록 한다.

In [9]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
print(torch.version.cuda)  # CUDA 버전 확인
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인

!export CUDA_LAUNCH_BLOCKING=1

# PyTorch에서 가중치 추출 및 Keras에 적용
pt_weights = {k: v.cpu().numpy() for k, v in weight.items()}

# Keras 가중치 설정
# pytorch의 모델 가중치는 (출력, 입력) 순서로 저장되고 케라스는 그 반대이므로 Transpose 필요
keras_model.get_layer('hidden1').set_weights([pt_weights['hidden.0.weight'].T]) # bias 빼기!
# keras_model.get_layer('hidden2').set_weights([pt_weights['hidden2.0.weight'].T]) # 레이어 1개만 사용하는 경우 빼야 함!
keras_model.get_layer('output').set_weights([pt_weights['layer.0.weight'].T])

# 잘 들어갔나 확인?
keras_model.summary()

2.4.1
11.8
True
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 모델 저장
keras_model.save('poisson_1000_one_layer.h5')

In [11]:
# 모델의 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[ 0.1393217  -0.11117818  0.40588138 ...  0.160218   -0.09410918
   0.4136175 ]
 [ 0.06481083 -0.01649775  0.15433973 ... -0.08817419 -0.4773201
   0.0384829 ]
 [-0.60930616 -0.12605259  0.3287924  ...  0.06578878  0.23608081
  -0.6416051 ]
 ...
 [-0.41282886  0.10655718 -0.00813517 ...  0.1971226  -0.09260934
   0.32584062]
 [-0.4444129   0.01996902 -0.0946975  ... -0.10874777  0.12241526
   0.34488752]
 [-0.49544457  0.05372631  0.07568143 ...  0.22207184  0.34606954
  -0.23418602]]
Layer: relu1
Layer: output
[[ 0.14234476 -0.25528443]
 [-0.8288827   0.6202623 ]
 [ 0.2360881  -0.0081592 ]
 ...
 [-0.3477605   0.4772581 ]
 [-1.9131286   1.7017852 ]
 [-2.237751    1.935989  ]]
Layer: output_relu


In [12]:
from tensorflow.keras.models import load_model

# 저장된 모델 로드
loaded_model = load_model('/home/hschoi/leehyunwon/ECG-SNN/to_keras_conversion/poisson_1000_one_layer.h5')


In [13]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 로드된 모델의 가중치 확인
for layer in loaded_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)


Layer: input
Layer: hidden1
[[ 0.1393217  -0.11117818  0.40588138 ...  0.160218   -0.09410918
   0.4136175 ]
 [ 0.06481083 -0.01649775  0.15433973 ... -0.08817419 -0.4773201
   0.0384829 ]
 [-0.60930616 -0.12605259  0.3287924  ...  0.06578878  0.23608081
  -0.6416051 ]
 ...
 [-0.41282886  0.10655718 -0.00813517 ...  0.1971226  -0.09260934
   0.32584062]
 [-0.4444129   0.01996902 -0.0946975  ... -0.10874777  0.12241526
   0.34488752]
 [-0.49544457  0.05372631  0.07568143 ...  0.22207184  0.34606954
  -0.23418602]]
Layer: relu1
Layer: output
[[ 0.14234476 -0.25528443]
 [-0.8288827   0.6202623 ]
 [ 0.2360881  -0.0081592 ]
 ...
 [-0.3477605   0.4772581 ]
 [-1.9131286   1.7017852 ]
 [-2.237751    1.935989  ]]
Layer: output_relu
